In [1]:
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from efficientnet_pytorch import EfficientNet
import os
import pandas as pd
from PIL import Image
from tqdm.auto import tqdm
from torch.optim.lr_scheduler import ExponentialLR
from torch.cuda.amp import autocast, GradScaler

# Parameters
IMG_SIZE = 244
PATH = 'C:/Users/Sri Ram/Untitled Folder 8/model_checkpoint.pth'  # Update with your desired path
NUM_CLASSES = 1000
NUM_WORKERS = 4
BATCH_SIZE = 32
LEARNING_RATE = 0.001
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
NUM_EPOCHS = 10

# Define the ImageDataset class
class ImageDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def load_image(self, img_path):
        img = Image.open(img_path).convert('RGB')
        img = img.resize((IMG_SIZE, IMG_SIZE))
        return img

    def __getitem__(self, idx):
        fname = self.df.loc[idx, 'filename']
        label = self.df.loc[idx, 'label']
        img_path = os.path.join(self.img_dir, fname)
        img = self.load_image(img_path)
        if self.transform:
            img = self.transform(img)  # Convert PIL to Tensor
        return img, label

# Load Dataframes
train_df = pd.read_csv('df_train.csv')

# Dataset & Dataloader
train_transform = transforms.Compose([
    transforms.ToTensor()])
train_ds = ImageDataset(train_df, 'D:/DATA/augmented_images', transform=train_transform)
train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=NUM_CLASSES).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scheduler = ExponentialLR(optimizer, gamma=0.1)

Loaded pretrained weights for efficientnet-b0


In [3]:

# Gradient scaling
scaler = GradScaler()

# Training loop
pbar = tqdm(total=NUM_EPOCHS, desc="Training")
for epoch in range(NUM_EPOCHS):
    scheduler.step()
    epoch_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    pbar_iterations = tqdm(total=len(train_dl), desc=f"Epoch {epoch + 1}/{NUM_EPOCHS}", leave=True)
    

    for X_batch, y_batch in train_dl:
        with autocast():
            
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_pred = model(X_batch)
            

            loss = torch.nn.functional.cross_entropy(y_pred, y_batch)
            epoch_loss += loss.item()         
            


        # Calculate accuracy
        _, predicted_labels = torch.max(y_pred, 1)
        correct_predictions += (predicted_labels == y_batch).sum().item()
        total_samples += y_batch.size(0)
        

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()
        pbar_iterations.update(1)
        
    pbar_iterations.close() 
    optimizer.step()
    scheduler.step()

    avg_loss = epoch_loss / len(train_dl)
    accuracy = correct_predictions / total_samples
    print(f'Epoch [{epoch+1}/{NUM_EPOCHS}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}')
    pbar.set_postfix(train_acc=accuracy)
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_loss,
            'accuracy': accuracy
     },PATH)

# Close the tqdm progress bar
pbar.close()

Training:   0%|          | 0/10 [00:00<?, ?it/s]

D:\anaconda_installed\envs\tfgputest\lib\site-packages\torch\optim\lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Epoch 1/10:   0%|          | 0/2160 [00:00<?, ?it/s]

Epoch [1/10], Loss: 0.6260, Accuracy: 0.8040


Epoch 2/10:   0%|          | 0/2160 [00:00<?, ?it/s]

Epoch [2/10], Loss: 0.1624, Accuracy: 0.9455


Epoch 3/10:   0%|          | 0/2160 [00:00<?, ?it/s]

KeyboardInterrupt: 